<a href="https://colab.research.google.com/github/jisung4984/JavaGithub/blob/main/%EA%B1%B0%EB%8C%80%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8_%EC%97%AC%EB%9F%AC%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers tokenizers
!pip install "transformers>=4.38.0" datasets accelerate sentencepiece


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 121.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [3]:
import pandas as pd

train_df = pd.read_csv("/content/mydrive/MyDrive/train.csv")
test_df  = pd.read_csv("/content/mydrive/MyDrive/test.csv")

print(train_df.columns)
print(test_df.columns)


Index(['title', 'full_text', 'generated'], dtype='object')
Index(['ID', 'title', 'paragraph_index', 'paragraph_text'], dtype='object')


In [4]:
def split_into_sentences(df, text_col, label_col=None, id_col=None):
    rows = []
    for _, row in df.iterrows():
        text = str(row[text_col])
        parts = [p.strip() for p in text.split("\n") if p.strip() != ""]
        for p in parts:
            sample = {"text": p}
            if label_col:
                sample["label"] = row[label_col]
            if id_col:
                sample["doc_id"] = row[id_col]
            rows.append(sample)
    return pd.DataFrame(rows)


In [5]:
train_sent_df = split_into_sentences(train_df, "full_text", label_col="generated")
test_sent_df  = split_into_sentences(test_df,  "paragraph_text", id_col="ID")

print("Train sentence samples:", len(train_sent_df))
print("Test sentence samples:",  len(test_sent_df))


Train sentence samples: 1226364
Test sentence samples: 1962


In [10]:
def smart_oversample(df, target_ratio=0.35, max_dup=3):
    human = df[df["label"] == 0]
    gpt   = df[df["label"] == 1]

    human_n = len(human)
    gpt_n   = len(gpt)

    print(f"\n📌 Original: human={human_n}, gpt={gpt_n} ({gpt_n/(human_n+gpt_n)*100:.2f}%)")

    # GPT 목표 비율
    desired_gpt_n = int(human_n * target_ratio / (1 - target_ratio))

    if desired_gpt_n <= gpt_n:
        print("GPT 이미 충분, oversampling 불필요.")
        return df

    # oversampling needed
    need = desired_gpt_n - gpt_n

    # 최대 중복 횟수 제한
    max_total = len(gpt) * max_dup
    need = min(need, max_total)

    # 중복 최소화하여 필요한 수만큼 샘플링
    gpt_oversampled = gpt.sample(
        n=need,
        replace=True
    )

    # 같은 문장이 max_dup 회수 이상 나오지 않게 제한
    gpt_oversampled = gpt_oversampled.groupby("text").head(max_dup)

    result = pd.concat([human, gpt, gpt_oversampled], ignore_index=True)
    result = result.sample(frac=1).reset_index(drop=True)

    print(f"➡ After Oversampling: human={len(human)}, gpt={len(result)-len(human)} "
          f"({(len(result)-len(human))/len(result)*100:.2f}%)")

    return result

train_sent_balanced = smart_oversample(train_sent_df, target_ratio=0.2, max_dup=2)



📌 Original: human=1125652, gpt=100712 (8.21%)
➡ After Oversampling: human=1125652, gpt=238481 (17.48%)


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_sent_balanced)
test_dataset  = Dataset.from_pandas(test_sent_df)


In [8]:
def build_tokenize_fn(tokenizer):
    def tokenize(batch):
        out = tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )
        if "label" in batch:
            out["labels"] = batch["label"]
        return out
    return tokenize


In [9]:
 from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch, gc, numpy as np

model_list = [
    "klue/roberta-base",
    "beomi/KcELECTRA-base",
    "monologg/kobert"
]

results = {}
trained_models = {}

for model_name in model_list:
    print(f"\n🚀 Training: {model_name}\n")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenize = build_tokenize_fn(tokenizer)

    tokenized = train_dataset.map(tokenize, batched=True)

    # split
    tokenized = tokenized.train_test_split(test_size=0.1)
    train_set = tokenized["train"]
    valid_set = tokenized["test"]

    train_set.set_format("torch", columns=["input_ids","attention_mask","labels"])
    valid_set.set_format("torch", columns=["input_ids","attention_mask","labels"])

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    training_args = TrainingArguments(
    output_dir=f"/content/{model_name.replace('/', '_')}",
    eval_strategy="epoch",        # ← 여기가 변경됨
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none"
)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set
    )

    trainer.train()
    eval_result = trainer.evaluate()

    results[model_name] = eval_result
    trained_models[model_name] = (trainer, tokenizer)

    print("⭐ Eval:", eval_result)

    del trainer, model
    torch.cuda.empty_cache()
    gc.collect()



🚀 Training: klue/roberta-base



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
best_model = max(results, key=lambda m: results[m]["eval_accuracy"])
best_trainer, best_tokenizer = trained_models[best_model]

print(f"\n🔥 Best model:", best_model)


In [ ]:
def tokenize_test(batch):
    return best_tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

test_tokenized = test_dataset.map(tokenize_test, batched=True)
test_tokenized.set_format("torch", columns=["input_ids","attention_mask"])


In [ ]:
pred = best_trainer.predict(test_tokenized)
sentence_probs = pred.predictions[:, 1]

test_sent_df["prob"] = sentence_probs

doc_pred = (
    test_sent_df.groupby("doc_id")["prob"]
    .max()  # 문장 중 가장 GPT스러운 문장
    .apply(lambda x: 1 if x >= 0.3 else 0)  # threshold=0.3
    .reset_index()
)

doc_pred.columns = ["ID", "label"]


In [ ]:
doc_pred.to_csv("/content/submission.csv", index=False)
print("\n🎉 submission.csv 생성 완료!")
